<a href="https://colab.research.google.com/github/taraumutoni/Machine_Learning_course_UGent_D012554_kaggle/blob/master/Eye_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import pandas as pd

In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")

testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")




In [0]:
trainset_copy = trainset.copy()
testset_copy = testset.copy()
label_column = trainset.pop('label')
index_column = testset.pop('index')


In [0]:
#1.Start of by weighing all features at the same scale
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np


scaler = StandardScaler()
scaler.fit(trainset)
trainset_scaled = scaler.transform(trainset)
testset_scaled = scaler.transform(testset)

In [22]:
trainset_scaled= pd.DataFrame(data=trainset_scaled)
trainset_scaled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.018351,-0.396476,0.677912,-0.277047,0.733699,0.167400,1.330668,0.454721,0.093570,-0.068336,0.418469,0.026714,-0.156365,0.252992
1,0.052070,-0.791842,-0.136210,0.346482,-0.434286,-0.259658,0.007585,-1.609301,-0.482085,-0.276922,-0.232997,0.235973,0.060820,-0.037576
2,0.574178,0.222036,0.061066,-0.002733,-0.496011,-0.373355,0.031954,-0.716795,-0.482085,-0.381214,-1.894254,0.392280,0.091976,0.322059
3,2.972354,3.177226,5.588146,5.335202,6.480134,7.221060,4.638614,6.311759,6.199439,6.193291,4.834001,5.588622,3.349453,2.812530
4,1.293521,-1.135609,0.135164,-0.975478,-0.496011,-0.117120,1.208346,-0.131037,0.554655,0.505527,1.911632,1.776139,1.782573,1.401236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-2.430832,0.222036,-1.616738,-0.725969,-0.311435,0.167400,-0.017261,-0.521542,-0.741551,-0.276922,-1.347334,-1.931461,-1.831536,-2.555566
1996,-0.864783,0.874724,-1.246726,-0.476460,0.272258,-0.060549,-0.286752,-0.660775,-0.453442,-0.824839,-0.485547,-1.409078,-0.559580,-1.033657
1997,-0.342950,-0.053044,0.159704,0.272067,-0.126858,-0.487052,-1.683418,-0.633038,-1.086943,-0.120228,-0.359272,-0.913170,-0.233802,-0.175981
1998,-0.060714,0.754439,0.579275,1.219520,1.440242,0.707600,1.011962,2.769472,0.611940,-0.120228,0.376650,0.053189,-0.032346,-0.521857


In [23]:
testset_scaled = pd.DataFrame(data=testset_scaled)
testset_scaled.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.103077,1.046608,-0.506702,0.072168,-0.004007,-0.088835,0.154276,-0.772271,-0.971812,-1.346811,-0.169859,-0.495161,-0.202948,-0.466549
1,-0.244195,-0.499673,-0.802616,-0.975478,-0.802839,-0.202532,-0.702932,0.398702,-0.136692,-0.146683,-0.443319,-0.913170,-0.358124,-0.590924
2,-0.018351,0.342321,0.282880,-0.301852,0.149407,0.849583,-0.237536,0.649974,0.583297,0.140249,0.040053,0.105122,-0.249531,-0.258808
3,-0.554489,-0.155906,-0.013033,-0.102439,-1.079704,-0.941841,-1.511881,-1.414048,-1.461540,-0.903186,-0.359272,-0.338853,0.107403,-0.425001
4,0.475423,-1.358756,-0.161230,-1.025087,-0.004007,-1.084378,-0.041630,-0.410047,-0.539931,0.035956,-0.085812,0.183531,-0.388978,0.349848


In [24]:
from sklearn import metrics
import seaborn as sns;
import matplotlib.pyplot as plt;


from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score 
import numpy as np

model = LogisticRegression(C=1) #we want to compute a logistic function that predicts well the label for our training points
model.fit(trainset_scaled,label_column)# fitting logistic regression on trainingset


#predictions = model.predict_proba(trainset_scaled)[:,1]

#print("R-squared = {}".format(metrics.r2_score(label_column,predictions)))

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
score_acc = cross_val_score(model, trainset_scaled, label_column, scoring='roc_auc', cv=10).mean() # measure the auc score 
print(score_acc)


0.6593382417786088


In [26]:
from sklearn.model_selection import GridSearchCV

search_space = [0.001,0.01,0.1,1,10,100] #C hyperparameter, previous model I used 1


model = LogisticRegression()
params = dict(C=search_space)
grid_search = GridSearchCV(model, param_grid=params) # model is estimator
grid_search.fit(trainset_scaled, label_column) # C-values overlopen op je trainingset 





print(grid_search.best_estimator_) 

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [27]:
model = LogisticRegression( C=100)
model.fit(trainset_scaled,label_column)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
predictions = model.predict_proba(testset_scaled)

print(predictions)


[[0.76329792 0.23670208]
 [0.64229297 0.35770703]
 [0.68605999 0.31394001]
 ...
 [0.52637569 0.47362431]
 [0.36303085 0.63696915]
 [0.57893101 0.42106899]]


In [29]:
model.predict(trainset_scaled)# predict the label on trainingset

array([1, 1, 1, ..., 0, 0, 1])

In [30]:
train_respons = model.predict(trainset_scaled)
len(train_respons)

2000

In [32]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

score_acc = accuracy_score(label_column,train_respons)# train accuracy

print(score_acc) #accuracy of true value vs the model prediction

0.632


In [0]:
prediction_test = pd.DataFrame(columns=['index','label'])
prediction_test['index'] = index_column
prediction_test['label'] = predictions[:,1] # all rows and column 1

 

filename = "Eye_prediction_Logistic_regression.csv"
prediction_test.to_csv(filename, index=False)